In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
from dataset_working import ImageNetDataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import PIL
import os
from pathlib import Path
from typing import Callable, List, Tuple, Dict
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import torch

import pandas as pd


import matplotlib.pyplot as plt

from torch import nn
import torch.optim as optim
import numpy as np
import tqdm as tqdm
import copy
from sklearn.metrics import f1_score
import shutil
from inspect import stack
from sklearn.preprocessing import label_binarize
from itertools import cycle

from torch.utils.data import Dataset, DataLoader
import PIL
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.model_selection import train_test_split
from finetuner import finetune_model

/home/danya-sakharov/AvanpostHackathon/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ood import find_smart_thresholds, test_model_ood
from finetuner import load_model_finetune, load_model
from dataset_working import get_dataloaders_train


data_dir = 'dataset'
classes = {'trucks':0, 'minibus':1, 'ski':2, 'dump_trucks':3, 'bicycles':4, 'snowboard':5, 'tractor':6, 'trains':7, 'gazon':8, 'horses':9}
data_dir ="/home/danya-sakharov/AvanpostHackathon/ml/dataset"
planes = "/home/danya-sakharov/AvanpostHackathon/ml/planes"

meta = dict()
np.random.seed(42)
torch.seed()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model_ft = load_model(classes, "/home/danya-sakharov/AvanpostHackathon/ml/model50.pth", device)
model_ft.eval()
dataloaders = get_dataloaders_train(data_dir, classes)

find_smart_thresholds(model_ft, device, dataloaders, test_model_ood)

/home/danya-sakharov/AvanpostHackathon/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/danya-sakharov/AvanpostHackathon/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn

{'init_val_f1': 0.948432284699526,
 'init_val_acc': tensor(0.9485),
 'init_test_f1': 0.980379209365589,
 'init_test_acc': tensor(0.9803),
 'best_threshold': [0.07051203,
  0.08375094,
  0.63695383,
  0.039096385,
  0.2678677,
  0.34997958,
  0.19771491,
  0.11990949,
  0.23373367,
  0.23180081],
 'val_accuracy': tensor(0.9691),
 'val_f1': 0.9694142444396204,
 'test_accuracy': tensor(0.9803),
 'test_f1': 0.980379209365589}

In [1]:
from finetuner import predict_samples, predict_samples_old
classes = {'trucks':0, 'minibus':1, 'ski':2, 'dump_trucks':3, 'bicycles':4, 'snowboard':5, 'tractor':6, 'trains':7, 'gazon':8, 'horses':9}
pth_path = "/home/danya-sakharov/AvanpostHackathon/ml/model50.pth"

new = predict_samples(classes, pth_path, "/home/danya-sakharov/AvanpostHackathon/ml/new_data/original")
old = predict_samples_old(classes, pth_path, "/home/danya-sakharov/AvanpostHackathon/ml/new_data/original")

/home/danya-sakharov/AvanpostHackathon/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new.values()

dict_values([5, 5, 5, 5, 5, 5, 5, 9, 7, 7, 5, 7, 9, 7, 5, 5, 5, 5, 5, 3, 9, 5, 5, 0, 2, 3, 9, 9, 5, 9, 5, 3, 5, 5, 5, 5, 5, 7, 0, -1, 5, 5, 5, 7, 3, 9, 5, 5, -1, 3, 9, 5, 5, 5, 9, 5, 9, 7, 5, 9, 5, 5, 7, 5, 7, 5, 7, 5, 9, 5, 5, 5, 5, 5, 9, 5, 7])

In [3]:
old.values()

dict_values([9, 5, 5, 0, 5, 5, 5, 5, 7, 5, 7, 9, 9, 9, 5, 5, 5, 7, 5, 5, 5, 5, 5, 9, 5, 5, 5, 3, 5, 5, 7, 9, 5, 5, 3, 9, 9, 5, 5, 3, 5, 5, 5, 9, 7, 5, 5, 9, 5, 5, 9, 7, 7, 9, 9, 5, 5, 5, 5, 5, 7, 7, 5, 9, 2, 5, 0, 7, 5, 9, 5, 5, 3, 5, 7, 3, 5])

In [2]:
data_dir = 'dataset'
classes = {'trucks':0, 'minibus':1, 'ski':2, 'dump_trucks':3, 'bicycles':4, 'snowboard':5, 'tractor':6, 'trains':7, 'gazon':8, 'horses':9}
data_dir ="/home/danya-sakharov/AvanpostHackathon/ml/dataset"
planes = "/home/danya-sakharov/AvanpostHackathon/ml/planes"
meta = finetune_model(data_dir, list(classes.keys()), 'model50.pth', planes ,"planes")
meta

Epoch 0/9
----------
train Loss: 1.6328 Acc: 0.6371
val Loss: 0.9499 Acc: 0.8852
Epoch 1/9
----------
train Loss: 0.7712 Acc: 0.9059
val Loss: 0.5615 Acc: 0.9373
Epoch 2/9
----------
train Loss: 0.5336 Acc: 0.9301
val Loss: 0.4206 Acc: 0.9346
Epoch 3/9
----------
train Loss: 0.4187 Acc: 0.9382
val Loss: 0.3592 Acc: 0.9382
Epoch 4/9
----------
train Loss: 0.3519 Acc: 0.9438
val Loss: 0.3108 Acc: 0.9505
Epoch 5/9
----------
train Loss: 0.3190 Acc: 0.9433
val Loss: 0.2753 Acc: 0.9505
Epoch 6/9
----------
train Loss: 0.2966 Acc: 0.9445
val Loss: 0.2785 Acc: 0.9488
Epoch 7/9
----------
train Loss: 0.2859 Acc: 0.9449
val Loss: 0.2526 Acc: 0.9435
Epoch 8/9
----------
train Loss: 0.2742 Acc: 0.9505
val Loss: 0.2565 Acc: 0.9549
Epoch 9/9
----------
train Loss: 0.2752 Acc: 0.9492
val Loss: 0.2439 Acc: 0.9585
Training complete in 7m 41s
Best val Acc: 0.958481


{'classes': ['trucks',
  'minibus',
  'ski',
  'dump_trucks',
  'bicycles',
  'snowboard',
  'tractor',
  'trains',
  'gazon',
  'horses',
  'planes'],
 'pth_path': PosixPath('/home/danya-sakharov/AvanpostHackathon/ml/model50_fine.pth'),
 'epoch_loss': [1.632797222405053,
  0.9499036941427224,
  0.7712286037783469,
  0.5615175360083159,
  0.5335773545400895,
  0.42056288114706114,
  0.4187101714388754,
  0.3591751686886427,
  0.3519143162972623,
  0.3107919912034968,
  0.31897824956577436,
  0.27530527141195305,
  0.29659038364650187,
  0.2785289649828584,
  0.28593354541487365,
  0.2525568639747667,
  0.27418092612244627,
  0.2565142015791613,
  0.275164556243859,
  0.24386713050911368],
 'epoch_acc': [0.6370967741935484,
  0.8851590106007067,
  0.9058771542200619,
  0.9372791519434629,
  0.9300707026071586,
  0.9346289752650176,
  0.9382456915598764,
  0.9381625441696113,
  0.943769332744145,
  0.950530035335689,
  0.9433274414494035,
  0.950530035335689,
  0.9445426425099426,
  0.94

In [5]:
import pandas as pd
import torch
from dataset_working import get_all_files, ImageNetDatasetPred, get_test_transforms
data_dir ="/home/danya-sakharov/AvanpostHackathon-1/ml/palnes"
NUM_WORKERS = 1
new_files, _ = get_all_files(data_dir=data_dir)
new_lables = 0 * len(new_files)
init_dataframe = pd.DataFrame({'x': new_files, 'y': new_lables})
dataset_pred = ImageNetDatasetPred(
    init_dataframe, transform=get_test_transforms())
dataloader_pred = torch.utils.data.DataLoader(
    dataset_pred, batch_size=64, shuffle=True, num_workers=NUM_WORKERS)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [6]:
new_files

[]

In [1]:
from finetuner import predict_samples
classes = {'trucks':0, 'minibus':1, 'ski':2, 'dump_trucks':3, 'bicycles':4, 'snowboard':5, 'tractor':6, 'trains':7, 'gazon':8, 'horses':9, 'planes':10}
d = predict_samples(list(classes.keys()), "model_fine.pth", "/home/danya-sakharov/AvanpostHackathon-1/ml/planes")

/home/danya-sakharov/AvanpostHackathon-1/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
